# IPL Auction Price Prediction Analysis !!!

### Project done by
### 1.) SHAIK AZHARUDDIN ( AM19M016 )
### 2. ) SHIVAM SINGH ( AM19M029 )

### Project Overview

The objective of this analysis is to predict the price of the players who are going to participate in IPL 2017. The model has to be trained seperately for batsman , bowler and all rounder. 

In [30]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression as lr
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold   #For K-fold cross validation
from sklearn import metrics
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier as DTC

In [4]:
# Reading processed data

batsman_2017 = pd.read_csv("C:/Users/HP/Desktop/resumes/shivam/Guided-projects-master/Guided-projects-master/IPL-auction-prediction-master/ProcessedData/batsman_2017.csv")
batsman_2016 = pd.read_csv("C:/Users/HP/Desktop/resumes/shivam/Guided-projects-master/Guided-projects-master/IPL-auction-prediction-master/ProcessedData/batsman_2016.csv")
batsman_2015 = pd.read_csv("C:/Users/HP/Desktop/resumes/shivam/Guided-projects-master/Guided-projects-master/IPL-auction-prediction-master/ProcessedData/batsman_2015.csv")
batsman_2014 = pd.read_csv("C:/Users/HP/Desktop/resumes/shivam/Guided-projects-master/Guided-projects-master/IPL-auction-prediction-master/ProcessedData/batsman_2014.csv")

baller_2017 = pd.read_csv("C:/Users/HP/Desktop/resumes/shivam/Guided-projects-master/Guided-projects-master/IPL-auction-prediction-master/ProcessedData/baller_2017.csv")
baller_2016 = pd.read_csv("C:/Users/HP/Desktop/resumes/shivam/Guided-projects-master/Guided-projects-master/IPL-auction-prediction-master/ProcessedData/baller_2016.csv")
baller_2015 = pd.read_csv("C:/Users/HP/Desktop/resumes/shivam/Guided-projects-master/Guided-projects-master/IPL-auction-prediction-master/ProcessedData/baller_2015.csv")
baller_2014 = pd.read_csv("C:/Users/HP/Desktop/resumes/shivam/Guided-projects-master/Guided-projects-master/IPL-auction-prediction-master/ProcessedData/baller_2014.csv")

allrounder_2017 = pd.read_csv("C:/Users/HP/Desktop/resumes/shivam/Guided-projects-master/Guided-projects-master/IPL-auction-prediction-master/ProcessedData/allrounder_2017.csv")
allrounder_2016 = pd.read_csv("C:/Users/HP/Desktop/resumes/shivam/Guided-projects-master/Guided-projects-master/IPL-auction-prediction-master/ProcessedData/allrounder_2016.csv")
allrounder_2015 = pd.read_csv("C:/Users/HP/Desktop/resumes/shivam/Guided-projects-master/Guided-projects-master/IPL-auction-prediction-master/ProcessedData/allrounder_2015.csv")
allrounder_2014 = pd.read_csv("C:/Users/HP/Desktop/resumes/shivam/Guided-projects-master/Guided-projects-master/IPL-auction-prediction-master/ProcessedData/allrounder_2014.csv")

In [7]:
batsman_2017.head(3)

,Player,Mat_t20,Inns_t20,NO_t20,HS_t20,Ave_t20,SR_t20,4s_t20,6s_t20,Team_t20,...,4s_odi,6s_odi,Team_odi,Runs_odi,100s_odi,50s_odi,0s_odi,Price,Score,Price_score
0,C Munro,14,14,2,109,47.66,181.58,44,40,DD,...,49.0,16.0,DD,430.0,0.0,3.0,2.0,190,271.58,190.000000
1,MJ Guptill,14,14,0,105,33.21,138.80,31,27,NaN,...,63.0,21.0,NaN,633.0,2.0,1.0,0.0,0,201.80,1771.272202
2,RG Sharma,12,12,0,118,26.25,172.13,34,18,NaN,...,135.0,52.0,NaN,1463.0,7.0,5.0,2.0,0,226.13,1984.825485


In [8]:
baller_2017.head(3)

,Player,Mat_t20,Inns_t20,Overs_t20,Mdns_t20,Runs_t20,BBI_t20,Ave_t20,Econ_t20,SR_t20,...,Ave_odi,Econ_odi,SR_odi,Team_odi,Wkts_odi,4s_odi,5s_odi,Price,Score,Price_score
0,TA Boult,12.0,12.0,43.5,0.0,382.0,1-Apr,20.10,8.71,13.8,...,26.56,5.35,29.7,NaN,32.0,1.0,2.0,0.0,22.51,911.172812
1,YS Chahal,10.0,10.0,37.3,0.0,343.0,23-Apr,21.43,9.14,14.0,...,23.29,4.97,28.0,NaN,37.0,1.0,1.0,0.0,23.14,886.365601
2,TG Southee,13.0,12.0,41.0,0.0,361.0,13-Mar,22.56,8.80,15.3,...,34.29,5.57,36.8,NaN,24.0,0.0,0.0,0.0,24.10,851.058091


In [9]:
## Setting training data
training_batsman = pd.concat([batsman_2014, batsman_2015, batsman_2016], ignore_index = True)
training_baller = pd.concat([baller_2014, baller_2015, baller_2016], ignore_index = True)
training_allrounder = pd.concat([allrounder_2014, allrounder_2015, allrounder_2016], ignore_index = True)

In [32]:
training_baller.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 321 entries, 0 to 320
Data columns (total 32 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   index           321 non-null    int64  
 1   Player          321 non-null    object 
 2   Mat_t20         321 non-null    int64  
 3   Inns_t20        321 non-null    int64  
 4   Overs_t20       321 non-null    float64
 5   Mdns_t20        321 non-null    int64  
 6   Runs_t20        321 non-null    int64  
 7   BBI_t20         321 non-null    object 
 8   Ave_t20         321 non-null    float64
 9   Econ_t20        321 non-null    float64
 10  SR_t20          321 non-null    float64
 11  Team_t20        321 non-null    object 
 12  Wkts_t20        321 non-null    int64  
 13  4s_t20          321 non-null    int64  
 14  5s_t20          321 non-null    int64  
 15  Mat_odi         321 non-null    float64
 16  Inns_odi        321 non-null    float64
 17  Overs_odi       321 non-null    flo

In [23]:
training_batsman.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 397 entries, 0 to 396
Data columns (total 32 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   index           397 non-null    int64  
 1   Player          397 non-null    object 
 2   Mat_t20         397 non-null    int64  
 3   Inns_t20        397 non-null    int64  
 4   NO_t20          397 non-null    int64  
 5   HS_t20          397 non-null    int64  
 6   Ave_t20         397 non-null    float64
 7   SR_t20          397 non-null    float64
 8   4s_t20          397 non-null    int64  
 9   6s_t20          397 non-null    int64  
 10  Team_t20        397 non-null    object 
 11  Runs_t20        397 non-null    int64  
 12  100s_t20        397 non-null    int64  
 13  50s_t20         397 non-null    int64  
 14  0s_t20          397 non-null    int64  
 15  Mat_odi         397 non-null    float64
 16  Inns_odi        397 non-null    float64
 17  NO_odi          397 non-null    flo

In [12]:
training_batsman.head()

,Player,Mat_t20,Inns_t20,NO_t20,HS_t20,Ave_t20,SR_t20,4s_t20,6s_t20,Team_t20,...,6s_odi,Team_odi,Runs_odi,100s_odi,50s_odi,0s_odi,Price,Score,Price_score,Price_adjusted
0,V Kohli,7,7,3,77,96.25,133.68,33,11,NaN,...,13.0,NaN,940.0,4.0,3.0,1.0,0,182.68,352.518263,2886.697693
1,AD Hales,9,9,1,116,44.12,144.08,34,15,NaN,...,1.0,NaN,126.0,0.0,0.0,1.0,0,195.08,376.446589,3082.641701
2,AJ Finch,10,10,1,71,33.11,130.70,37,9,NaN,...,14.0,NaN,810.0,3.0,4.0,2.0,0,178.70,344.838043,2823.805987
3,F du Plessis,7,7,1,119,47.33,179.74,29,12,NaN,...,16.0,NaN,940.0,3.0,6.0,1.0,0,222.74,429.822192,3519.723255
4,JP Duminy,11,10,4,86,44.00,120.54,19,10,NaN,...,10.0,NaN,505.0,1.0,2.0,0.0,0,150.54,290.497588,2378.823466


In [13]:
training_batsman.isnull().sum()

Player              0
Mat_t20             0
Inns_t20            0
NO_t20              0
HS_t20              0
Ave_t20             0
SR_t20              0
4s_t20              0
6s_t20              0
Team_t20          385
Runs_t20            0
100s_t20            0
50s_t20             0
0s_t20              0
Mat_odi            88
Inns_odi           88
NO_odi             88
HS_odi             88
Ave_odi            88
SR_odi             88
4s_odi             88
6s_odi             88
Team_odi          388
Runs_odi           88
100s_odi           88
50s_odi            88
0s_odi             88
Price               0
Score               0
Price_score         0
Price_adjusted      0
dtype: int64

In [14]:
training_baller.isnull().sum()

Player              0
Mat_t20             0
Inns_t20            0
Overs_t20           0
Mdns_t20            0
Runs_t20            0
BBI_t20             0
Ave_t20             0
Econ_t20            0
SR_t20              0
Team_t20          305
Wkts_t20            0
4s_t20              0
5s_t20              0
Mat_odi            82
Inns_odi           82
Overs_odi          82
Mdns_odi           82
Runs_odi           82
BBI_odi            82
Ave_odi            82
Econ_odi           82
SR_odi             82
Team_odi          311
Wkts_odi           82
4s_odi             82
5s_odi             82
Price               0
Score               0
Price_score         0
Price_adjusted      0
dtype: int64

In [15]:
training_allrounder.isnull().sum()

Player              0
Runs_t20            0
HS_t20              0
Bat Av_t20          0
Wkts_t20            0
BBI_t20             0
Bowl Av_t20         0
Ct_t20              0
St_t20              0
Ave Diff_t20        0
Team_t20          460
Matches_t20         0
100s_t20            0
5s_t20              0
Runs_odi          113
HS_odi            113
Bat Av_odi        113
Wkts_odi          113
BBI_odi           113
Bowl Av_odi       113
Ct_odi            113
St_odi            113
Ave Diff_odi      113
Team_odi          465
Matches_odi       113
100s_odi          113
5s_odi            113
Price               0
Score               0
Price_score         0
Price_adjusted      0
dtype: int64

In [21]:
training_batsman = training_batsman[training_batsman['Price_adjusted'] > 0].reset_index()
training_baller = training_baller[training_baller['Price_adjusted'] > 0].reset_index()
training_allrounder = training_allrounder[training_allrounder['Price_adjusted'] > 0].reset_index()

training_batsman_y = training_batsman['Price_adjusted']
training_batsman_x = training_batsman.drop(['Price', 'Player', 'Price_adjusted', 'Price_score', 'Score', 'Team_t20', 'Team_odi'], axis = 1)
training_baller_y = training_baller['Price_adjusted']
training_baller_x = training_baller.drop(['Price', 'Player', 'Price_adjusted', 'Price_score', 'Score', 'Team_t20', 'Team_odi', 'BBI_t20', 'BBI_odi'], axis = 1)
training_allrounder_y = training_allrounder['Price_adjusted']
training_allrounder_x = training_allrounder.drop(['Price', 'Player', 'Price_adjusted', 'Price_score', 'Score', 'Team_t20', 'Team_odi', 'BBI_t20', 'BBI_odi'], axis = 1)

training_batsman.fillna(0,inplace=True)
training_baller.fillna(0,inplace=True)
training_allrounder.fillna(0,inplace=True)

## Setting test data
test_batsman = batsman_2017
test_baller = baller_2017
test_allrounder = allrounder_2017

test_batsman = test_batsman[test_batsman['Price_score'] > 0].reset_index()
test_baller = test_baller[test_baller['Price_score'] > 0].reset_index()
test_allrounder = test_allrounder[test_allrounder['Price_score'] > 0].reset_index()
test_batsman['Price_adjusted'] = test_batsman['Price_score']
test_baller['Price_adjusted'] = test_baller['Price_score']
test_allrounder['Price_adjusted'] = test_allrounder['Price_score']

test_batsman_y = test_batsman['Price']
test_batsman_x = test_batsman.drop(['Price', 'Player', 'Price_score', 'Price_adjusted', 'Score', 'Team_t20', 'Team_odi'], axis = 1)
test_baller_y = test_baller['Price_score']
test_baller_x = test_baller.drop(['Price', 'Player', 'Price_score', 'Price_adjusted', 'Score', 'Team_t20', 'Team_odi', 'BBI_t20', 'BBI_odi'], axis = 1)
test_allrounder_y = test_allrounder['Price_score']
test_allrounder_x = test_allrounder.drop(['Price', 'Player', 'Price_score', 'Price_adjusted', 'Score', 'Team_t20', 'Team_odi', 'BBI_t20', 'BBI_odi'], axis = 1)

test_batsman.fillna(0,inplace=True)
test_baller.fillna(0,inplace=True)
test_allrounder.fillna(0,inplace=True)

In [43]:
def model_building(model, predictors, outcome, data, test_data):
    '''Function to build model, cross-validate and predict results'''
    #model.fit(data[predictors], data[outcome])  
    kf = KFold(n_splits = 3)
    error = []
    for train, test in kf.split(data[predictors],data[outcome]):
        train_predictors = (data[predictors].iloc[train,:])
        train_target = data[outcome].iloc[train]
        model.fit(train_predictors, train_target)
        error.append(model.score(data[predictors].iloc[test,:], data[outcome].iloc[test]))
    cv_error = np.mean(np.array(error)**2)
    #print('Cross validation Score : %s' % '{0:.3%}'.format(cv_error))
    model.fit(data[predictors],data[outcome])
    #coefficients = [model.intercept_, model.coef_]
    #print coefficients
    predictions = model.predict(test_data[predictors])
    predictions = [predictions[i][0] for i in range(len(predictions))]
    test_data["predicted_price"] = predictions
    #print test_data["predicted_price"]
    accuracy = np.mean(np.subtract(test_data["predicted_price"].values,test_data[outcome].values)**2)
    #print('Accuracy : %s' % '{0:.3%}'.format(accuracy))
    #test_data["Team1"] = test_data["team1"].apply(remap)
    #test_data["Team2"] = test_data["team2"].apply(remap)
    #test_data["Actual Winner"] = test_data["winner"].apply(remap)
    #test_data["Predicted Winner"] = test_data["predicted_winner"].apply(remap)
    #df = test_data[["Team1","Team2","Actual Winner", "Predicted Winner"]]
    return [accuracy, cv_error]

In [44]:
# Models tested
model1 = lr()                         #linear regression
'''model2 = LogisticRegression()         #L2 regularization, one vs all
model3 = LogisticRegression(penalty='l1')         #L1 regularization, one vs all
model4 = LogisticRegression(solver='newton-cg', multi_class='multinomial')  #Multinomial
model5 = SVC(kernel = "linear")
model6 = DTC()
model7 = RandomForestClassifier(n_estimators=100)
'''
models = [model1]#, model2, model3, model4, model5, model6, model7]
#results = []
accuracies = []
cv_errors = []
#col = ["Intercept"] + list(train_data.columns)
#coefficients_summary = pd.DataFrame(columns= col)
#Batsman
output = ['Price_adjusted']
predictors = training_batsman.drop(['Price', 'Player', 'Price_score', 'Price_adjusted', 'Score', 'Team_t20', 'Team_odi'],1).columns
for model in models:
    [accuracy, cv_error] = model_building(model, predictors, output, training_batsman, test_batsman)
    #results.append(result)
    accuracies.append(accuracy)
    cv_errors.append(cv_error)
    #coefficients_summary = coefficients_summary.append(coefficients, ignore_index=True)
model_names = ["Linear Regression"]#, "LogisticRegression(One vs All) L2 reg", "LogisticRegression(One vs All) L1 reg", "MultinomialRegression", "SVM", "DecisionTree", "Random Forest"]
model_comparison = pd.DataFrame(columns=["Model Names", "MSE", "Cross Validation Errors"])
model_comparison["Model Names"] = model_names
model_comparison["MSE"] = accuracies
model_comparison["Cross Validation Errors"] = cv_errors

In [45]:
model_comparison

,Model Names,MSE,Cross Validation Errors
0,Linear Regression,1.036084e+06,0.27962


In [47]:
model1 = lr()
model1.fit(training_batsman[predictors],training_batsman[output])
#coefficients = [model.intercept_, model.coef_]
#print coefficients
predictions = model.predict(test_batsman[predictors])
predictions = [predictions[i][0] for i in range(len(predictions))]

In [48]:
predictions

[3728.110995723675,
 2633.94845574023,
 3062.6256059976204,
 2557.021566387184,
 2366.739877358319,
 2226.9210958884496,
 2520.8851750262115,
 2579.0318628027244,
 2443.2112012666666,
 2347.4107437396806,
 2480.120483187582,
 2326.447035125095,
 2610.1732675794974,
 1892.3991598972725,
 2505.3278053129193,
 2077.069563167581,
 2425.819816798773,
 2599.488189599576,
 2097.989445532342,
 2097.0601350466313,
 2203.7761382879467,
 2369.9661204605486,
 2132.788973292024,
 2106.5734958780463,
 2360.5676976680065,
 2108.5898339553005,
 1989.7620304347752,
 2312.2002744988827,
 2363.9038610373213,
 1847.7573664036447,
 1929.9585938969644,
 2268.3623563082533,
 1859.4858725870922,
 2523.6776873825575,
 2118.007214009535,
 1710.9498122612426,
 1591.9522853223323,
 2510.449083508089,
 1862.609968877807,
 2072.6168457891654,
 1950.0507914195475,
 2205.6898268485793,
 2219.6579570915974,
 2216.5492101092377,
 2099.1847145932024,
 1896.2685466981668,
 1709.4465544637799,
 2146.9623571115826,
 2491.3